<div align="center">

# **Exploracion y limpieza de los siniestros**

</div>

## Librerias

In [271]:
library(readxl)
library(lubridate)

In [272]:
source("../src/data/ajuste_dinero_ipc.R")

## Data

In [273]:
siniestros <- read_excel("../data/input/Siniestros_Hist.xlsx")

In [274]:
colnames(siniestros)

[1] "FECHASIN"         "VLRPRIMAPAG"      "VLRPRISUSCR"      "VLRASEGU"        
 [5] "VLRASEGU_ORIG"    "VLRPAGADO."       "VLRSININCUR"      "VLRDEDUCIBLE"    
 [9] "VLRRECOBRO"       "VLRRSVACONSTIAMP" "VLRRSVAPAGAMP"    "FECPAGOAMP"      
[13] "ESTADO_FINAL"     "COBERTURA_FINAL"

In [275]:
names(siniestros) <- c(
  "Fecha",
  "Prima_efectivamente_pagada_hasta_fecha_siniestro",
  "Prima_suscrita_incluida_cobertura_responsabilidad_civil",
  "Asegurado_otras_coberturas_distintas_responsabilidad_civil",
  "Asegurado_original",
  "Efectivamente_pagado_por_siniestro",
  "Siniestro_incurrido",
  "Deducible",
  "Recobro_obtenido",
  "Reserva_constituida_para_amparo",
  "Reserva_pagada_para_amparo",
  "Fecha_pago_amparo",
  "Estado_final_siniestro",
  "Cobertura"
)

In [276]:
unique(siniestros$Cobertura)

[1] "PTH"       "PPD"       "RC BIENES" "PPH"       "RC PERS"

In [277]:
siniestros$Cobertura <- factor(siniestros$Cobertura,
                               levels = c("PTH", "PPD", "RC BIENES", "PPH", "RC PERS"),
                               labels = c("pth", 
                                          "ppd",
                                          "rc",
                                          "pph",
                                          "rc"))

Juntamos ambas responsabilidades civiles para que concuerde con las coberturas de polizas_v2.txt

## Fechas

In [278]:
siniestros$Fecha <- as.Date(siniestros$Fecha)

In [279]:
unique(format(siniestros$Fecha, "%Y"))

[1] "2017" "2018"

In [280]:
unique(format(siniestros$Fecha[year(siniestros$Fecha) == 2017], "%m"))

[1] "01"

In [281]:
unique(format(siniestros$Fecha[year(siniestros$Fecha) == 2018], "%m"))

[1] "01" "02" "03" "04" "05" "06" "07" "08" "09" "10" "11" "12"

Vamos a ignorar 2017 porque solo hay datos de enero

In [282]:
siniestros <- siniestros[year(siniestros$Fecha) == 2018, ]

## Nulos

In [283]:
sum(is.na(siniestros))

[1] 5380

In [284]:
colSums(is.na(siniestros))

Fecha 
                                                         0 
          Prima_efectivamente_pagada_hasta_fecha_siniestro 
                                                         0 
   Prima_suscrita_incluida_cobertura_responsabilidad_civil 
                                                         0 
Asegurado_otras_coberturas_distintas_responsabilidad_civil 
                                                         0 
                                        Asegurado_original 
                                                         0 
                        Efectivamente_pagado_por_siniestro 
                                                         0 
                                       Siniestro_incurrido 
                                                         0 
                                                 Deducible 
                                                         0 
                                          Recobro_obtenido 
                                                         0 
                           Reserva_constituida_para_amparo 
                                                         0 
                                Reserva_pagada_para_amparo 
                                                         0 
                                         Fecha_pago_amparo 
                                                      5380 
                                    Estado_final_siniestro 
                                                         0 
                                                 Cobertura 
                                                         0

Fecha_pago_amparo no es relevante para la modelacion que estamos llevando a cabo

## Eliminando duplicados

In [285]:
siniestros <- unique(siniestros)

## Seleccion de columnas

Vamos a seleccionar Siniestro_incurrido como la severidad, pues es el valor neto de los siniestros sin haber descontado el pago del deducible u otras posibles alteraciones del valor de cada accidente

In [286]:
siniestros <- siniestros[, c("Fecha", "Siniestro_incurrido", "Cobertura")]
names(siniestros) <- c("fecha", "severidad", "cobertura")
siniestros$semana <- week(siniestros$fecha)
siniestros$dia <- yday(siniestros$fecha)

## Llevando la severidad a enero de 2019

In [287]:
siniestros <- ajuste_dinero_ipc(siniestros, "fecha", "severidad")

## Segmentando por cobertura y eliminado valores extraños

- pph

In [288]:
siniestros_pph <- siniestros[(siniestros$cobertura) == "pph", ]
head(sort(unique(siniestros_pph$severidad)),20)

[1]      0.00  35245.61  41120.40  48747.69  64963.42  84273.51  89809.13
 [8] 115978.22 116139.55 142873.71 148001.55 158309.19 161411.76 162588.43
[15] 167265.75 171020.88 173384.72 178223.05 179107.02 194332.18

In [289]:
nrow(siniestros_pph[(siniestros_pph$severidad < 70000),])*100/nrow(siniestros_pph)

[1] 14.88498

In [290]:
siniestros_pph <- siniestros_pph[(siniestros_pph$severidad >= 70000),]

- pth

In [291]:
siniestros_pth <- siniestros[(siniestros$cobertura) == "pth", ]
head(sort(unique(siniestros_pth$severidad)),20)

[1]      0.00  98013.54 116282.09 117180.52 160672.58 168071.23 201423.28
 [8] 279360.22 300007.33 312625.65 347420.97 347837.87 347906.76 348776.53
[15] 352879.00 353715.49 355943.90 378353.92 382935.47 385347.96

In [292]:
nrow(siniestros_pth[(siniestros_pth$severidad < 3000000),])*100/nrow(siniestros_pth)

[1] 21.60883

In [293]:
siniestros_pth <- siniestros_pth[(siniestros_pth$severidad >= 3000000),]

- ppd

In [294]:
siniestros_ppd <- siniestros[(siniestros$cobertura) == "ppd", ]
head(sort(unique(siniestros_ppd$severidad)),20)

[1]     0.000  1724.491  3430.260  3941.680  5907.670  9412.643 14352.093
 [8] 14636.996 16161.209 20491.290 23296.270 24189.512 27606.212 33332.915
[15] 37942.136 39237.076 43948.893 44300.778 45189.162 46270.689

In [295]:
nrow(siniestros_ppd[(siniestros_ppd$severidad < 70000),])*100/nrow(siniestros_ppd)

[1] 8.921866

In [296]:
siniestros_ppd <- siniestros_ppd[(siniestros_ppd$severidad >= 70000),]

- rh

In [297]:
siniestros_rc <- siniestros[(siniestros$cobertura) == "rc", ]
head(sort(unique(siniestros_rc$severidad)),20)

[1]      0.00  19868.00  22486.97  30910.40  69359.54  73845.87  75315.27
 [8]  75526.30  75616.93  76352.57  83579.59  89083.77 100785.39 107481.00
[15] 117587.33 122164.11 122856.11 123784.91 123939.16 127099.54

In [298]:
nrow(siniestros_rc[(siniestros_rc$severidad < 500000),])*100/nrow(siniestros_rc)

[1] 18.6086

In [299]:
siniestros_rc <- siniestros_rc[(siniestros_rc$severidad >= 500000),]

## Agrupando por semanas y dia

[1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
[109] 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
[127] 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
[145] 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
[163] 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
[181] 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
[199] 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
[217] 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
[235] 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252
[253] 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270
[271] 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288
[289] 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306
[307] 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324
[325] 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342
[343] 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360
[361] 361 362 363 364 365

## Exportando resultados